## 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# user_metadata= pd.read_csv('')
# product_metadata= pd.read_csv('')

## 일단 여긴 예시데이터로

In [ ]:
# 예시 final_df (프롬프트와 목표평점이 이미 존재한다고 가정)
final_df = pd.DataFrame({
    'prompts': [
        "사용자 정보: 나이: 25, 목표평점: 1,고용 상태: Employed, 결혼 여부: Married, 주택 소유 여부: Own, 교육 수준: Graduate, 파산 기록: No, 이전 대출 연체 여부: No, 신용 점수: 750, 대출 금액: 20000 원\n대출 상품 정보: 상품 유형: 주택 담보 대출, 최대 대출 한도: 500000000 원, 요구 신용 점수: 700, 이자율: 3.5%\n질문: 이 사용자는 이 대출 상품에 대해 승인될까요? (1: 승인, 0: 미승인)",
        "사용자 정보: 나이: 32이고 목표평점은 0입니다., 고용 상태: Unemployed, 결혼 여부: Single, 주택 소유 여부: Rent, 교육 수준: Undergraduate, 파산 기록: Yes, 이전 대출 연체 여부: Yes, 신용 점수: 600, 대출 금액: 10000 원\n대출 상품 정보: 상품 유형: 전세 자금 대출, 최대 대출 한도: 100000000 원, 요구 신용 점수: 650, 이자율: 4.0%\n질문: 이 사용자는 이 대출 상품에 대해 승인될까요? (1: 승인, 0: 미승인)",
        # 다른 프롬프트들...
    ],
    'TargetScore': [1, 0]  # 목표 평점
})

##이거!!!최종!!

In [ ]:
import google.generativeai as genai
import os

import re

# Gemini API 키 설정
genai.configure(api_key=GOOGLE_API_KEY)

# 모델 초기화
model = genai.GenerativeModel('gemini-pro')

# 목표평점에 대한 패턴을 정의
target_score_pattern = r'목표평점\s*[:\-]?\s*\d+(\.\d+)?'  # '목표평점: 90', '목표평점 85' 형태의 텍스트 마스킹
# 만약 앞에가 마침표가 아닌 연결어일 때 사용
#target_score_pattern = r'목표평점[^0-1]*\d'

# 생성된 응답을 저장할 리스트
responses = []

for prompt in final_df['prompts']:
    # 목표평점이 포함된 문장이 있다면 마스킹
    masked_prompt = re.sub(target_score_pattern, " ", prompt)

    # 모델을 통해 응답 생성
    response = model.generate_content(masked_prompt)

    # 응답의 텍스트 부분을 가져오고 불필요한 공백 제거
    response_text = response.text.strip()

    if response_text == "승인":
        responses.append(1)  # "승인"은 1로 변환
    elif response_text == "미승인":
        responses.append(0)  # "미승인"은 0으로 변환
    elif response_text == "1":
        responses.append(1)  # "1"은 그대로 1로 변환
    elif response_text == "0":
        responses.append(0)  # "0"은 그대로 0으로 변환
    else:
        responses.append(None)  # 그 외에는 None으로 처리

# 응답을 데이터프레임에 추가
final_df['generated_responses'] = responses

# 'TargetScore'와 'generated_responses' 컬럼을 문자열로 변환 => 나중에 비교할 때 같은 type으로 만들어주기 위함
final_df['TargetScore'] = final_df['TargetScore'].astype(str)
final_df['generated_responses'] = final_df['generated_responses'].astype(str)


In [ ]:
masked_prompt1 = re.sub(target_score_pattern, "", final_df['prompts'][0])
masked_prompt2 = re.sub(target_score_pattern, "", final_df['prompts'][1])
print(masked_prompt1 ,masked_prompt2 )

사용자 정보: 나이: 25, ,고용 상태: Employed, 결혼 여부: Married, 주택 소유 여부: Own, 교육 수준: Graduate, 파산 기록: No, 이전 대출 연체 여부: No, 신용 점수: 750, 대출 금액: 20000 원
대출 상품 정보: 상품 유형: 주택 담보 대출, 최대 대출 한도: 500000000 원, 요구 신용 점수: 700, 이자율: 3.5%
질문: 이 사용자는 이 대출 상품에 대해 승인될까요? (1: 승인, 0: 미승인) 사용자 정보: 나이: 32이고 입니다., 고용 상태: Unemployed, 결혼 여부: Single, 주택 소유 여부: Rent, 교육 수준: Undergraduate, 파산 기록: Yes, 이전 대출 연체 여부: Yes, 신용 점수: 600, 대출 금액: 10000 원
대출 상품 정보: 상품 유형: 전세 자금 대출, 최대 대출 한도: 100000000 원, 요구 신용 점수: 650, 이자율: 4.0%
질문: 이 사용자는 이 대출 상품에 대해 승인될까요? (1: 승인, 0: 미승인)


### 마스킹에 대하여

만약 프롬프트가 목표평점에 대한 언급을 하게되면 이를 마스킹해야함.

1. 최대한 앞, 뒤 정보와 컴마로 구분하도록 하기
2. 연결어로 설명한다면 패턴을 바꿔야함.

# 최종 결과 데이터셋 구성

In [ ]:
# TargetScore와 generated_responses 값이 같은 데이터만 추출
filtered_df = final_df[final_df['TargetScore'] == final_df['generated_responses']]

# 필터링된 데이터프레임 출력
filtered_df

# 결과를 파일로 저장 (선택사항)
# filtered_df.to_csv('filtered_results.csv', index=False)



,prompts,TargetScore,generated_responses
0,"사용자 정보: 나이: 25, 목표평점: 1,고용 상태: Employed, 결혼 여부...",1,1
1,"사용자 정보: 나이: 32이고 목표평점은 0입니다., 고용 상태: Unemploye...",0,0


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 정확도 계산
accuracy = accuracy_score(final_df['TargetScore'], final_df['generated_responses'])
print(f"Accuracy: {accuracy:.4f}")

# 혼동 행렬 출력
conf_matrix = confusion_matrix(final_df['TargetScore'], final_df['generated_responses'])
print("Confusion Matrix:")
print(conf_matrix)

# 분류 성능 보고서
report = classification_report(final_df['TargetScore'], final_df['generated_responses'])
print("Classification Report:")
print(report)
